# 0. Environment Setting & Read data

In [19]:
import pandas as pd

In [20]:
order = pd.read_csv('C:/Users/happy/Desktop/project/data/order.csv', index_col=0)
order_last = pd.read_csv('C:/Users/happy/Desktop/project/data/order_last.csv',index_col=0)

In [21]:
order.head(10)

,ID,alight_station,off_station,adult,teen,child,distance,fare,suc_status,suc_status_d,call_time,alight_time,off_time,suc_time,cancel_time,x,y,alight_adm,off_adm
0,1,10258,10093,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:24,NaN,NaN,NaN,2021-02-15 00:00:25,37.508302,126.513957,운북동,운남동
1,7,10258,12005,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:35,NaN,NaN,NaN,2021-02-15 00:00:35,37.508586,126.513748,운북동,운서동(주거)
2,1,10258,10093,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:46,NaN,NaN,NaN,2021-02-15 00:00:46,37.508371,126.514027,운북동,운남동
3,7901,10258,10196,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:47,NaN,NaN,NaN,2021-02-15 00:00:47,37.508612,126.513747,운북동,운북동
4,7,10258,12005,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:56,NaN,NaN,NaN,2021-02-15 00:00:56,37.508598,126.513748,운북동,운서동(주거)
5,11067,10046,10101,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:59,NaN,NaN,NaN,2021-02-15 00:00:59,37.508559,126.513665,운북동,중산동
6,1,10258,10093,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:59,NaN,NaN,NaN,2021-02-15 00:00:59,37.508506,126.513753,운북동,운남동
7,5,10263,10101,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:01:02,NaN,NaN,NaN,2021-02-15 00:01:02,37.508697,126.513936,운북동,중산동
8,6,10258,10046,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:01:12,NaN,NaN,NaN,2021-02-15 00:01:12,37.508609,126.513719,운북동,운북동
9,11067,10046,10101,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:01:25,NaN,NaN,NaN,2021-02-15 00:01:25,37.508559,126.513665,운북동,중산동


In [22]:
# column 별 null 값 갯수
order.isnull().sum()

ID                      0
alight_station          0
off_station             0
adult                   0
teen                    0
child                   0
distance                0
fare                    0
suc_status              0
suc_status_d      3612594
call_time               0
alight_time       3523346
off_time          3523346
suc_time          3487874
cancel_time        129908
x                    4843
y                    4843
alight_adm              0
off_adm                 2
dtype: int64

In [23]:
# datatype conversion 
order['call_time'] = order['call_time'].astype('datetime64')
order['alight_time'] = order['alight_time'].astype('datetime64')
order['off_time'] = order['off_time'].astype('datetime64')
order['suc_time'] = order['suc_time'].astype('datetime64')

# 1. 

### Call Time

In [24]:
print(min(order['call_time']))
print(max(order['call_time']))

2021-02-15 00:00:24
2022-07-02 22:59:05


### Alight / Off Time

In [25]:
print(min(order[order['alight_time'].notnull()]['alight_time']))
print(max(order[order['alight_time'].notnull()]['alight_time']))

2021-02-15 00:09:11
2022-07-02 23:13:50


In [26]:
print(min(order[order['off_time'].notnull()]['off_time']))
print(max(order[order['off_time'].notnull()]['off_time']))

2021-02-15 00:10:58
2022-07-02 23:23:14


In [27]:
# 파생변수 ivt 생성
order['ivt'] = order['off_time'] - order['alight_time']
order.head(3)

,ID,alight_station,off_station,adult,teen,child,distance,fare,suc_status,suc_status_d,call_time,alight_time,off_time,suc_time,cancel_time,x,y,alight_adm,off_adm,ivt
0,1,10258,10093,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:24,NaT,NaT,NaT,2021-02-15 00:00:25,37.508302,126.513957,운북동,운남동,NaT
1,7,10258,12005,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:35,NaT,NaT,NaT,2021-02-15 00:00:35,37.508586,126.513748,운북동,운서동(주거),NaT
2,1,10258,10093,1,0,0,0.0,0,배차불가,NaN,2021-02-15 00:00:46,NaT,NaT,NaT,2021-02-15 00:00:46,37.508371,126.514027,운북동,운남동,NaT


In [28]:
# 나중에 지우기
print(min(order[order['ivt'].notnull()]['ivt']))
print(max(order[order['ivt'].notnull()]['ivt']))

0 days 00:00:01
0 days 17:09:18
